In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [5]:
df = pd.read_csv("../input/fake-news/train.csv")

In [6]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


**Data Preprocessing**

In [11]:
##Droping Nan Values
df = df.dropna()

In [12]:
y = df['label']

In [17]:
X = df.drop('label', axis = 1)

In [18]:
X

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [19]:
X.shape

(18285, 4)

In [20]:
import tensorflow as tf

In [22]:
tf.__version__

'2.4.1'

In [23]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [57]:
##Vocabulary Size
voca = 5000

**One Hot Representation**

In [25]:
messages = X.copy()

In [26]:
messages.reset_index(inplace = True)

In [28]:
import nltk
import re
from nltk.corpus import stopwords

In [29]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [58]:
# Data Preprocessing

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
print('Done')

Done


In [59]:
##One Hot Representation
onehot_repr = [one_hot(words,voca) for words in corpus]
print('Done')

Done


**Embedding Representation**

In [60]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding = 'pre',maxlen = sent_length)

In [61]:
len(embedded_docs)

18285

In [82]:
##Creating Model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voca,embedding_vector_features,input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [65]:
len(embedded_docs)

18285

In [66]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [68]:
##Splitting the Model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.2, random_state = 45)

**Model Training**

In [83]:
model.fit(X_train, y_train,validation_data = (X_test, y_test),epochs = 10, batch_size = 64)

Epoch 1/10
229/229 [==============================] - 9s 33ms/step - loss: 0.4675 - accuracy: 0.7555 - val_loss: 0.1816 - val_accuracy: 0.9188
Epoch 2/10
229/229 [==============================] - 7s 30ms/step - loss: 0.1350 - accuracy: 0.9487 - val_loss: 0.1820 - val_accuracy: 0.9202
Epoch 3/10
229/229 [==============================] - 7s 30ms/step - loss: 0.0941 - accuracy: 0.9651 - val_loss: 0.1989 - val_accuracy: 0.9207
Epoch 4/10
229/229 [==============================] - 7s 30ms/step - loss: 0.0702 - accuracy: 0.9759 - val_loss: 0.2305 - val_accuracy: 0.9188
Epoch 5/10
229/229 [==============================] - 7s 30ms/step - loss: 0.0507 - accuracy: 0.9817 - val_loss: 0.2681 - val_accuracy: 0.9098
Epoch 6/10
229/229 [==============================] - 7s 30ms/step - loss: 0.0396 - accuracy: 0.9881 - val_loss: 0.2867 - val_accuracy: 0.9081
Epoch 7/10
229/229 [==============================] - 7s 30ms/step - loss: 0.0223 - accuracy: 0.9929 - val_loss: 0.3250 - val_accuracy: 0.9177

**Adding Dropout**

In [52]:
from tensorflow.keras.layers import Dropout
##Creating Model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voca,embedding_vector_features,input_length = sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

**Performance Metrics and Accuracy**

In [70]:
y_pred = model.predict_classes(X_test)


/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [71]:
from sklearn.metrics import confusion_matrix

In [72]:
confusion_matrix(y_test,y_pred)

array([[1924,  177],
       [ 137, 1419]])

In [73]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9141372709871479

**TEST**

In [74]:
test = pd.read_csv('../input/fake-news/test.csv')

In [75]:
messages_test = test.copy()

In [76]:
#Data Processing
corpus_test = []
for i in range(0, len(messages_test)):
    
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus_test.append(review)
print('Done')

Done


In [77]:
##One Hot Representation
onehot_repr = [one_hot(words,voca) for words in corpus_test]
print('Done')

Done


In [78]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding = 'pre',maxlen = sent_length)

In [79]:
len(embedded_docs)

5200

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [81]:
x_test=np.array(embedded_docs)

In [84]:
pred=model.predict_classes(x_test)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [86]:
submission_data = pd.read_csv('/kaggle/input/fake-news/submit.csv')

In [87]:
submission_data['label']=pred
submission_data.to_csv('submit.csv' ,index=False)